In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))  # go up one level

import pandas as pd
import numpy as np
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

from models import get_base_models, make_child_model
from utils.resampling_registry import get_resamplers
from utils.battle_logger import BattleLogger
from utils.breed_and_battle import breed_and_battle, breed_and_battle_with_population

In [2]:
import os
import glob
import pandas as pd

# 📂 Load all datasets with baseline PR AUC
def load_all_datasets(folder="./synthetic_datasets"):
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    datasets = []

    for f in files:
        df = pd.read_csv(f)
        X = df.drop("rare_event", axis=1)
        y = df["rare_event"]
        baseline_pr_auc = y.mean()

        datasets.append({
            "name": os.path.basename(f),
            "X": X,
            "y": y,
            "baseline_pr_auc": baseline_pr_auc
        })

        print(f"📊 {os.path.basename(f)} — Baseline PR AUC: {baseline_pr_auc:.3f}")

    return datasets

# Now load + sort
datasets = load_all_datasets()
datasets = sorted(datasets, key=lambda d: d["baseline_pr_auc"])

📊 baseline_easy_w5_n5_d0.csv — Baseline PR AUC: 0.055
📊 high_drift_w3_n10_d60.csv — Baseline PR AUC: 0.036
📊 imbalanced_sparse_w0_n10_d10.csv — Baseline PR AUC: 0.011
📊 mixed_realistic_w4_n15_d30.csv — Baseline PR AUC: 0.045
📊 noisy_overlap_w2_n20_d0.csv — Baseline PR AUC: 0.026


In [ ]:
from sklearn.model_selection import train_test_split

# 📊 Sort datasets by baseline PR AUC (ascending difficulty)
datasets = sorted(datasets, key=lambda d: d["baseline_pr_auc"])

survivors = get_base_models()
finalists = []

# 🌊 Stage-by-stage tournament
for stage_idx, dataset in enumerate(datasets):
    dataset_name = dataset["name"]
    print(f"\n🌊 Stage {stage_idx + 1}: {dataset_name}")

    X_train, X_test, y_train, y_test = train_test_split(
        dataset["X"], dataset["y"], stratify=dataset["y"], test_size=0.3, random_state=42
    )

    resamplers = get_resamplers(X_train, y_train, target_col="rare_event")
    resampled_datasets = {name: fn() for name, fn in resamplers.items()}

    # ✅ Update dataset metadata on all survivors
    for s in survivors:
        s["dataset_name"] = dataset_name
        s["baseline_pr_auc"] = dataset["baseline_pr_auc"]

    with BattleLogger(
        to_file=f"logs/battle_stage_{stage_idx + 1}.txt",
        js_file=f"logs/battle_stage_{stage_idx + 1}.js",
        inject_html=True,
        html_template="battle_template.html",
        html_output=f"logs/battle_stage_{stage_idx + 1}.html"
    ):
        survivors = breed_and_battle_with_population(
            model_population=survivors,
            resampled_datasets=resampled_datasets,
            X_test=X_test,
            y_test=y_test,
            generations=1,
            top_k=3,
            dataset_name=dataset_name,
            baseline_pr_auc=dataset["baseline_pr_auc"],
            debug=True
        )

    finalists = survivors  # Keep track of final survivors

    # 🔁 Rebuild base model configs from survivors for next stage
    survivors = [
        {
            "name": s["label"].split(" + ")[0],  # recover model name
            "model": make_child_model(s["params"]),
            "params": s["params"]
        }
        for s in survivors
    ]


🌊 Stage 1: imbalanced_sparse_w0_n10_d10.csv
5


/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiabo


🌊 Stage 2: noisy_overlap_w2_n20_d0.csv
51

🌊 Stage 3: high_drift_w3_n10_d60.csv
465

🌊 Stage 4: mixed_realistic_w4_n15_d30.csv
4191


In [ ]:
# 🏆 Final leaderboard
# print(f"\n🏆 Finalists after all rounds: {[m['label'] for m in finalists]}")

# print("\n🏆 Finalists after all rounds:")
# for i, model in enumerate(finalists, 1):
#     print(f"{i}. {model['label']}_G{model['generation']} | PR AUC: {model['score']['pr_auc']:.3f} | Lineage: {model['lineage']}")

In [ ]:
import pandas as pd

# 🧾 Build summary table from finalists
final_summary = []
for model in finalists:
    pr_auc = model['score']['pr_auc']
    baseline = model.get('baseline_pr_auc', 0.0)
    final_summary.append({
        "Label": f"{model['label']}_G{model['generation']}",
        "Generation": model['generation'],
        "PR AUC": round(pr_auc, 3),
        "Lineage": model.get("lineage", "—"),
        "Beats Baseline?": "✅ Yes" if pr_auc > baseline else "❌ No",
        "Baseline PR AUC": round(baseline, 3),
        "Dataset": model.get("dataset_name", "unknown")
    })

# df_final = pd.DataFrame(final_summary).sort_values(by=["PR AUC"], ascending=False)
df_final = pd.DataFrame(final_summary)

# 📊 Display final summary
print("\n📋 Finalist Leaderboard:")
print(df_final.to_string(index=False))

# 💾 Optionally save
df_final.to_csv("logs/finalists_summary.csv", index=False)

In [ ]:
df_final.head(30)